In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
df = pd.read_csv('insurance.csv')
df.tail()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Charger les données
df = pd.read_csv('insurance.csv')
df.head()


In [ ]:
# Séparer les caractéristiques des étiquettes
X = df.drop('expenses', axis=1)
y = df['expenses']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifier les colonnes numériques et catégorielles
num_features = ['age', 'bmi', 'children']
cat_features = ['sex', 'smoker', 'region']

# Créer un transformateur pour les colonnes catégorielles et numériques
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ])

# Créer un pipeline avec le préprocesseur et le modèle de régression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Entraîner le modèle
model.fit(X_train, y_train)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Créer un nouveau pipeline avec un modèle plus complexe
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Définir une grille de paramètres pour la recherche par grille
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}

# Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés par la recherche par grille
print(grid_search.best_params_)

# Utiliser le meilleur modèle trouvé par la recherche par grille
best_model = grid_search.best_estimator_

# Prédire les coûts de santé sur l'ensemble de test avec le meilleur modèle
y_pred = best_model.predict(X_test)

# Calculer l'erreur absolue moyenne
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

# Vérifier si l'erreur absolue moyenne est inférieure à 3500
assert mae < 3500, "L'erreur absolue moyenne doit être inférieure à 3500 pour réussir le défi."

# Visualiser les résultats
plt.scatter(y_test, y_pred)
plt.xlabel('Vraies valeurs')
plt.ylabel('Valeurs prédites')
plt.title('Vraies valeurs vs valeurs prédites')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.show()


In [ ]:
# Remplacez 'test_dataset' et 'test_labels' par 'X_test' et 'y_test' respectivement
# TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
loss, mae, mse = best_model.evaluate(X_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = best_model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
